In [1]:
!pip install kafka-python
!pip install psycopg2-binary

     |████████████████████████████████| 246 kB 662 kB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 3.4 MB/s eta 0:00:01


In [44]:
import psycopg2
from kafka import KafkaConsumer
from json import loads
import time
from pyspark.sql import SparkSession
from multiprocessing import Process, Manager
import json

In [45]:
consumer = KafkaConsumer('ny_tripdata',
                         bootstrap_servers=['docker-compose_kafka_1:29092'],
                         auto_offset_reset='earliest',
                         group_id='group')

In [46]:
manager = Manager()
data_queue = manager.Queue()
data_queue.qsize()

0

In [47]:
def get_most_relevant_feature(msg):
    data = {}
    data['vendor_name'] = msg['vendor_name']
    data['Trip_Pickup_DateTime'] = msg['Trip_Pickup_DateTime']
    data['Trip_Dropoff_DateTime'] = msg['Trip_Dropoff_DateTime']
    data['Trip_Distance'] = msg['Trip_Distance']
    data['Start_Lon'] = msg['Start_Lon']
    data['Start_Lat'] = msg['Start_Lat']
    data['End_Lon'] = msg['End_Lon']
    data['End_Lat'] = msg['End_Lat']
    return data

In [48]:
def insert_trip(conn,
                cur,
                vendor_name, 
                Trip_Pickup_DateTime, 
                Trip_Dropoff_DateTime,
                Trip_Distance,
                Start_Lon,
                Start_Lat,
                End_Lon,
                End_Lat):

    sql = """INSERT INTO trips (vendor_name, 
                                Trip_Pickup_DateTime, 
                                Trip_Dropoff_DateTime, 
                                Trip_Distance, 
                                Start_Lon, 
                                Start_Lat, 
                                End_Lon, 
                                End_Lat)
             VALUES(%s,%s,%s,%s,%s,%s,%s,%s);"""
    
    # execute the INSERT statement
    cur.execute(sql, (vendor_name, 
                      Trip_Pickup_DateTime, 
                      Trip_Dropoff_DateTime,
                      Trip_Distance,
                      Start_Lon,
                      Start_Lat,
                      End_Lon,
                      End_Lat,))
    # get the generated id back
    # vendor_id = cur.fetchone()[0]
    # commit the changes to the database
    conn.commit()

In [49]:
def consumeData(consumer, data_queue):
    try:
        # connect to the PostgreSQL database
        conn = psycopg2.connect(host="docker-compose_db_1",
                                database="postgres",
                                user="postgres",
                                password="postgres1234")
        # create a new cursor
        cur = conn.cursor()
        
        for message in consumer:
            try:
                msg = json.loads(message.value.decode("utf-8"))
                data = get_most_relevant_feature(msg)
                insert_trip(conn, cur, *list(data.values()))
                data_queue.put(data)
                print("Progress: {}".format(data_queue.qsize()), end="\r", flush=True)
            except Exception as e:
                print(data)
                print(e)
                pass
        # close communication with the database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

t1 = Process(target=consumeData, args=(consumer, data_queue, ))

In [50]:
t1.start()

{'vendor_name': 'CMT', 'Trip_Pickup_DateTime': '2009-01-30 20:13:', 'Trip_Dropoff_DateTime': None, 'Trip_Distance': None, 'Start_Lon': None, 'Start_Lat': None, 'End_Lon': None, 'End_Lat': None}
null value in column "trip_dropoff_datetime" of relation "trips" violates not-null constraint
DETAIL:  Failing row contains (15848, CMT, 2009-01-30 20:13:00, null, null, null, null, null, null, null).



In [53]:
data_queue.qsize()

15749

In [33]:
t1.terminate()
del t1